# SIMPLE CHARACTER CLASSIFICATION

## Import Library

import libraries that will be used in the code

In [ ]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

## Load Dataset

loads an image dataset so it can be used for training models & get class name

In [ ]:
dataset = os.path.join('..', 'Data')

classes = os.listdir(dataset)
classes.sort()

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

split the dataset into train & validation to train the model

In [ ]:
img_size = (75, 75)

train_generator = datagen.flow_from_directory(
    dataset,
    target_size=img_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset,
    target_size=img_size,
    class_mode='categorical',
    subset='validation'
)

classnames = list(train_generator.class_indices.keys())

## Build Model

Make a CNN model

In [ ]:
jumlah_kelas = train_generator.num_classes

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=train_generator.image_shape)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)


predictions = Dense(jumlah_kelas, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

## Train Model

early stopping of training if losses increase 3 times in a row

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights =True)

training model with train & validation dataset

In [ ]:
history = model.fit(
    train_generator,
    validation_data = validation_generator, 
    epochs = 100,
    callbacks = early_stopping
)

check training performance to see increases in accuracy

In [ ]:
epoch_nums = range(0,len(history.epoch))
training_loss = history.history["accuracy"]
validation_loss = history.history["val_accuracy"]
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

check training performance to see a decrease in loss

In [ ]:
epoch_nums = range(0,len(history.epoch))
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()